In [2]:
import glob as glob
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
from pathlib import Path

# Create the figures directory:

Path("figures").mkdir(parents=True, exist_ok=True)

In [3]:
# Data preprocessing
drone = pd.read_json('data/data_20092023-15:36:05(more_movement).json')
gcs = pd.read_json('data/data_20092023-15:39:26(less_movement).json')

# add prefix to packet type
drone['mavpackettype'] = 'UAV_' + drone['mavpackettype']
gcs['mavpackettype'] = 'GCS_' + gcs['mavpackettype']

# zero time to the beginning of the flight
drone['time_boot_ms'] -= drone['time_boot_ms'].min()
gcs['time_boot_ms'] -= gcs['time_boot_ms'].min()

# combine dataframes
df = pd.concat([drone, gcs], ignore_index=True)
df = df.sort_values(by=['time_boot_ms'], ignore_index=True)

df['time_s'] = df['time_boot_ms'] / 1000
df['lat'] = df['lat'] / 1e7 # int->float
df['lon'] = df['lon'] / 1e7
df['alt'] = df['alt'] / 1000 # orginally in mm

df['vx'] = df['vx'] / 100 # to m/s
df['vy'] = df['vy'] / 100
df['vz'] = -1*(df['vz'] / 100) # Invert z velocity. Up is now positive.

df = df[
    df['mavpackettype'].str.contains('UAV_GLOBAL_POSITION_INT')
]
df = df[['time_s', 'lat', 'lon', 'alt', 'vx', 'vy', 'vz']]
df.set_index('time_s', inplace=True)
df.index.name = 'time'
df

,lat,lon,alt,vx,vy,vz
time,,,,,,
0.140,42.724676,-84.480802,274.12,-0.01,-0.02,-0.00
0.473,42.724676,-84.480802,274.12,0.00,-0.02,-0.00
0.806,42.724676,-84.480802,274.12,0.00,-0.02,-0.00
1.138,42.724676,-84.480802,274.11,0.00,-0.02,-0.00
1.471,42.724675,-84.480802,274.11,0.00,-0.02,-0.00
...,...,...,...,...,...,...
75.398,42.724638,-84.480762,275.50,1.22,-0.39,-0.20
75.731,42.724641,-84.480763,275.55,0.92,-0.65,-0.00
76.063,42.724644,-84.480766,275.55,0.89,-0.89,-0.01


This is the data we are working with in this exmaple.

    - time is in seconds
    - vx: m/s in north direction
    - vy: m/s in east direction
    - vz: m/s upwards
    - alt: meters above sea level
    - lat and lon: geodesic coordinates

Next, use `pymap3d.geodetic2enu()` to convert lat, lon, and alt to east, north, up (m). The reference point is the starting point.

In [4]:
import pymap3d as pm

def geodesic_distances(lat1, lon1, alt1, lat2, lon2, alt2):
    # In mavlink, east is y, north is x, down is z (z flipped upwards in preprocessing)
    y, x, z = pm.geodetic2enu(lat1, lon1, alt1, lat2, lon2, alt2)
    return x, y, z

origin = df.iloc[0]

df['x'], df['y'], df['z'] = geodesic_distances(
    df['lat'], df['lon'], df['alt'], origin['lat'], origin['lon'], origin['alt']
)

df

,lat,lon,alt,vx,vy,vz,x,y,z
time,,,,,,,,,
0.140,42.724676,-84.480802,274.12,-0.01,-0.02,-0.00,0.000000,0.000000,0.000000e+00
0.473,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.011109,-0.008191,-2.696653e-10
0.806,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.022218,-0.016382,6.995199e-10
1.138,42.724676,-84.480802,274.11,0.00,-0.02,-0.00,-0.033328,-0.032763,-9.999998e-03
1.471,42.724675,-84.480802,274.11,0.00,-0.02,-0.00,-0.044437,-0.040954,-9.999999e-03
...,...,...,...,...,...,...,...,...,...
75.398,42.724638,-84.480762,275.50,1.22,-0.39,-0.20,-4.154847,3.276307,1.379998e+00
75.731,42.724641,-84.480763,275.55,0.92,-0.65,-0.00,-3.821570,3.137064,1.429998e+00
76.063,42.724644,-84.480766,275.55,0.89,-0.89,-0.01,-3.588277,2.907722,1.429998e+00


Run the incoming data through a moving average filter, in this case implemented via convolution. The goal of this is to remove random noise brought on by error in the GPS data. In the future, this should probably be implemented via a IIR filter for speed. To enable this functionality, set the `avg_flag` to `True`. You can also specify the pole count, which is the number of values included in the averaging operation. 

The problem with this implementation is there are issues with outliers in the boundaries. The higher the pole count, the more outliers in the last few values in the data will be present. This can be fixed with an IIR implementation.

In [5]:
def moving_average(x, n=3):
    return np.convolve(x, np.ones(n), 'same') / n

avg_flag = True

if avg_flag:

    # Take moving average of input data:

    n = 4

    df['x'] = moving_average(df['x'], n)
    df['y'] = moving_average(df['y'], n)
    df['z'] = moving_average(df['z'], n)

df

,lat,lon,alt,vx,vy,vz,x,y,z
time,,,,,,,,,
0.140,42.724676,-84.480802,274.12,-0.01,-0.02,-0.00,-0.002777,-0.002048,-6.741632e-11
0.473,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.008332,-0.006143,1.074636e-10
0.806,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.016664,-0.014334,-2.499999e-03
1.138,42.724676,-84.480802,274.11,0.00,-0.02,-0.00,-0.027773,-0.024572,-4.999999e-03
1.471,42.724675,-84.480802,274.11,0.00,-0.02,-0.00,-0.038882,-0.034811,-4.999999e-03
...,...,...,...,...,...,...,...,...,...
75.398,42.724638,-84.480762,275.50,1.22,-0.39,-0.20,-4.354813,3.307023,1.384998e+00
75.731,42.724641,-84.480763,275.55,0.92,-0.65,-0.00,-4.024314,3.167779,1.397498e+00
76.063,42.724644,-84.480766,275.55,0.89,-0.89,-0.01,-3.732697,2.985535,1.429998e+00


Calculate the prediction for the next timestamp using the current position and velocity.

In `df`, `x_pred`, `y_pred`, and `z_pred` are the current timestamp's predcitions for the next timestamp.

In `df_shifted`, the `*_pred` columns hold last timestamp's predction for this timestamp.

`df_shifted` is better for computing error metrics, and `df` is better for plotting.


In [6]:
# dt is the time to the next timestamp
df['dt'] = df.index.to_series().diff().shift(-1)

# Our guess for the position at the next timestamp is the current position plus the velocity times the time to the next timestamp
df['x_pred'] = df['x'] + df['vx'] * df['dt']
df['y_pred'] = df['y'] + df['vy'] * df['dt']
df['z_pred'] = df['z'] + df['vz'] * df['dt']

# now we need to shift the predictions forward one
# df_shifted now has the prediction for the current timestamp from the last timestamp
df_shifted = df.copy()
df_shifted['x_pred'] = df['x_pred'].shift(1)
df_shifted['y_pred'] = df['y_pred'].shift(1)
df_shifted['z_pred'] = df['z_pred'].shift(1)

,lat,lon,alt,vx,vy,vz,x,y,z,dt,x_pred,y_pred,z_pred
time,,,,,,,,,,,,,
0.140,42.724676,-84.480802,274.12,-0.01,-0.02,-0.00,-0.002777,-0.002048,-6.741632e-11,0.333,-0.006107,-0.008708,-6.741632e-11
0.473,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.008332,-0.006143,1.074636e-10,0.333,-0.008332,-0.012803,1.074636e-10
0.806,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.016664,-0.014334,-2.499999e-03,0.332,-0.016664,-0.020974,-2.499999e-03
1.138,42.724676,-84.480802,274.11,0.00,-0.02,-0.00,-0.027773,-0.024572,-4.999999e-03,0.333,-0.027773,-0.031232,-4.999999e-03
1.471,42.724675,-84.480802,274.11,0.00,-0.02,-0.00,-0.038882,-0.034811,-4.999999e-03,0.335,-0.038882,-0.041511,-4.999999e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75.398,42.724638,-84.480762,275.50,1.22,-0.39,-0.20,-4.354813,3.307023,1.384998e+00,0.333,-3.948553,3.177153,1.318398e+00
75.731,42.724641,-84.480763,275.55,0.92,-0.65,-0.00,-4.024314,3.167779,1.397498e+00,0.332,-3.718874,2.951979,1.397498e+00
76.063,42.724644,-84.480766,275.55,0.89,-0.89,-0.01,-3.732697,2.985535,1.429998e+00,0.333,-3.436327,2.689165,1.426668e+00


### Results
The table below is our guess(es) for the position at the next stamp(s). I believe these are the numbers that we'd like to feed the gimbal. 

In [19]:
df

,lat,lon,alt,vx,vy,vz,x,y,z,dt,x_pred,y_pred,z_pred,error
time,,,,,,,,,,,,,,
0.140,42.724676,-84.480802,274.12,-0.01,-0.02,-0.00,-0.002777,-0.002048,-6.741632e-11,0.333,-0.006107,-0.008708,-6.741632e-11,0.339502
0.473,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.008332,-0.006143,1.074636e-10,0.333,-0.008332,-0.012803,1.074636e-10,0.883255
0.806,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.016664,-0.014334,-2.499999e-03,0.332,-0.016664,-0.020974,-2.499999e-03,1.194209
1.138,42.724676,-84.480802,274.11,0.00,-0.02,-0.00,-0.027773,-0.024572,-4.999999e-03,0.333,-0.027773,-0.031232,-4.999999e-03,1.167133
1.471,42.724675,-84.480802,274.11,0.00,-0.02,-0.00,-0.038882,-0.034811,-4.999999e-03,0.335,-0.038882,-0.041511,-4.999999e-03,1.192415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75.398,42.724638,-84.480762,275.50,1.22,-0.39,-0.20,-4.354813,3.307023,1.384998e+00,0.333,-3.948553,3.177153,1.318398e+00,10.992914
75.731,42.724641,-84.480763,275.55,0.92,-0.65,-0.00,-4.024314,3.167779,1.397498e+00,0.332,-3.718874,2.951979,1.397498e+00,4.871658
76.063,42.724644,-84.480766,275.55,0.89,-0.89,-0.01,-3.732697,2.985535,1.429998e+00,0.333,-3.436327,2.689165,1.426668e+00,7.218311


### Error Analysis

In [7]:
error = 100 * np.sqrt( # euclidean distance - centimeters
    (df_shifted['x_pred'] - df_shifted['x'])**2 +
    (df_shifted['y_pred'] - df_shifted['y'])**2 +
    (df_shifted['z_pred'] - df_shifted['z'])**2
)  
error

time
0.140           NaN
0.473      0.339502
0.806      0.883255
1.138      1.194209
1.471      1.167133
            ...    
75.398     3.556635
75.731    10.992914
76.063     4.871658
76.396     7.218311
76.731    89.139121
Length: 231, dtype: float64

In [8]:
fig = px.histogram(error, nbins=50, marginal="violin") # or violin,)
fig.update_layout(
    title_text='Error in Position Prediction',
    xaxis_title_text='Error (cm)',
    yaxis_title_text='Count',
    bargap=0.2,
    bargroupgap=0.1
)

In [9]:
ninefifthpercent = np.quantile(error.dropna(), 0.95)
print(f'95% of the error is less than {ninefifthpercent:.2f} cm')

95% of the error is less than 12.19 cm


In [10]:
av = error.mean()
print(f'Average error is {av:.2f} cm')

Average error is 6.95 cm


In [11]:
sd = error.std()
print(f'Standard Deviation is: {sd:.2f} cm')

Standard Deviation is: 6.36 cm


In [12]:
below_15 = (error < 8).sum() / len(error)
print(f'{below_15:.2%} of the error is below 15cm')

67.97% of the error is below 15cm


Below are plots showing the locations in the telemetry streams (points), and their predicted movement to the next timestamp (lines).

In a perfect system, the lines would connect to the position at the next timestamp.

In [13]:
df['error'] = error.shift(-1)

fig = px.scatter(df, x='x', y='y')

fig.update_xaxes(title_font_family="Trebuchet")
fig.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1),
                  template = "plotly_white",
                  title="<b>Kinematic Model of Telemetry Stream</b>",
                 )
fig.update_layout(xaxis = dict(autorange="reversed"))


arrows = []
for index, row in df.iloc[3:-3].iterrows():
    arrow = go.layout.Annotation(dict(
                    x=row['x_pred'],
                    y=row['y_pred'],
                    xref="x", yref="y",
                    text="",
                    showarrow=True,
                    axref="x", ayref='y',
                    ax=row['x'],
                    ay=row['y'],
                    arrowhead=3,
                    arrowwidth=1.5,
                    arrowcolor='rgb(255,51,0)',)
                )
    arrows.append(arrow)


fig.update_layout(annotations=arrows)
# update the axes labels
fig.update_xaxes(title_text="East (m)")
fig.update_yaxes(title_text="North (m)")
# add subtitle
fig.add_annotation(xref='paper', yref='paper', x=0.5, y=1.05,
            text="Location (Blue) with Predicted Location for Next Timestamp (Red)",
            showarrow=False, font=dict(size=14))

fig.write_html("figures/2D-kinematic-model.html")
fig.show()

In [14]:
fig = go.Figure()

fig.update_xaxes(title_font_family="Trebuchet")
fig.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1),
                  template = "plotly_white",
                  title="<b>Kinematic Model of Telemetry Stream</b>",
                 )
fig.update_layout(xaxis = dict(autorange="reversed"))

fig.add_trace(go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z'],
    mode='markers',
    marker=dict(
        size=2,
        color=df['error'],                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
))
# add a line to from each point to the predicted point
for index, row in df.iterrows():
    fig.add_trace(go.Scatter3d(
        x=[row['x'], row['x_pred']],
        y=[row['y'], row['y_pred']],
        z=[row['z'], row['z_pred']],
        mode='lines',
        line=dict(color='red', width=1)
    ))
# remove the legend
fig.update_layout(showlegend=False)
fig.update_layout(scene = dict(
                    xaxis_title='East (m)',
                    yaxis_title='North (m)',
                    zaxis_title='Altitude (m)'))
fig.update_layout(
    title_text='Kinematic Model of Telemetry Stream',
    title_x=0.5,
)
# subtitle
fig.add_annotation(xref='paper', yref='paper', x=0.5, y=1.05,
            text="Location (Blue) with Predicted Location for Next Timestamp (Red)",
            showarrow=False, font=dict(size=14))

fig.write_html("figures/3D-kinematic-model.html")
fig.show()

# What's Going on in Z?

In [15]:
px.scatter(df, x=df.index, y='vz').show()

In [16]:
signal_series = df[df.index > 8].vz
signal = signal_series.values

sampling_rate = 1 / signal_series.index.diff().dropna().values.mean()
print(f'Sample rate: {sampling_rate:.2f} Hz')

Sample rate: 3.00 Hz


In [17]:
import numpy as np
from scipy.fft import fft, rfft
from scipy.fft import fftfreq, rfftfreq
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

In [18]:
fourier = fft(signal)

# Calculate N/2 to normalize the FFT output
N = len(signal)

# Plot the results
# plt.plot(rfftfreq(N, d=1/sampling_rate), 2*np.abs(rfft(signal))/N)
# plt.title('Spectrum')
# plt.xlabel('Frequency[Hz]')
# plt.ylabel('Amplitude')
# plt.show()

fig = px.line(x=rfftfreq(N, d=1/sampling_rate), y=2*np.abs(rfft(signal))/N)

fig.update_xaxes(title_font_family="Trebuchet")
fig.update_layout(template = "plotly_white",
                  title="<b>Fourier Transform of vz</b>",
                 )
fig.update_layout(
    xaxis_title_text='Frequency (Hz)',
    yaxis_title_text='Amplitude',
    bargap=0.2,
    bargroupgap=0.1
)
fig.write_html("figures/fourier-transform.html")
fig.show()